In [ ]:
#2000 emails sent to email_history took 5 minutes
%load_ext autoreload
%autoreload 2
import logging
import warnings

# Mute pandas UserWarning
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

logging.basicConfig(filename='Email_Scraper.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)

# Mute pandas UserWarning
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")


import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from Scraping_Emails.modules.scrapes import scrape
from Scraping_Emails.modules.db_operations import DatabaseConnector
from SQL_Scripts.email_failures import *
from update_bad_emails import *
from datetime import datetime, timedelta

email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'


# ----------------------------------Updating Faulty Emails in the DB before Send----------------------------------
#Searches GMAIL on the daily for bad emails that were sent back, and sends data to unsubscribed email table
#Goes through all distinct subject lines in the DB

db_connector_email_history = DatabaseConnector(server, database, db_username, db_password, db, 'email_history')
query = db_connector_email_history.SQL_query('SELECT DISTINCT subject FROM [emailcampaign].[dbo].[email_history]')
    
#Iterate through all unique subject lines to update the db table with all faulty email addresses
for line in query['subject']:
    print('\n')
    print(line)
    gather_and_send_bad_emails_to_db(line, email_address, email_pass)

# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass, start_date):

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame
    email_address = 'team@customplanet.com'
    email_pass = imap_password_customplanet
 
    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass, start_date)
    inbox = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass, start_date)
    outbox = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox['date'] = pd.to_datetime(outbox['date'], format='%Y-%m-%d')
    outbox.name = 'outbox'
        
    if inbox.empty == True:
        thread = pd.DataFrame()
        pass
    else: #this is a left merge on outbox
        thread = scrape.piece_together(outbox, inbox)
        thread = scrape.assign_sentiment(thread)
       
    # #If message was accidentally triggered more than once
    thread = thread.drop_duplicates(subset = ['subject', 'to'], keep='last')
    thread.reset_index(drop = True, inplace = True)
    outbox = scrape.map_reply_outbox(thread, outbox)


    return(inbox, outbox, thread)

#Get inbox, outbox, and create thread with given subject line back to given date
subject_line = 'Official'
inbox, outbox, thread= process(subject_line, email_address, email_pass, '11/15/2023')


# Instantiate the DatabaseConnector class, send over all emails in outbox variable to email_history table that contain the subject line 
#Optional column identifiers to add in to the DB
outbox['email_campaign_tag'] = 'Baseball Special Majestic'
outbox['sport'] = 'Baseball'    #frame   #table_name
db_connector_email_history.send(outbox, 'email_history')


if thread.empty != True:
    db_connector_thread = DatabaseConnector(server, database, db_username, db_password, db, 'thread')
    db_connector_thread.send(thread, 'thread')
    pre_existing_thread_updates = db_connector_thread.update_reply_thread(thread)
else:
    logging.info('Thread is an empty frame')
    print('Thread is an empty frame')

# To DO

In [ ]:
#Create PNGs on hoodies images using Selenium. 

#Write directly to MySQL DB

#Refine NLP

#Thirdly, what if the email is not directly replied to. In that case. 
# The scenario would be the following, email campaign sent out to sammytaylor2006@yahoo.com
#The customer indirectly reaches out to CP, and not through the thread. This calls for a seperate table.
# and a seperate scrape 

#Integrate SSIS at some point for views
#Integrating prior orders and linking them in a SQL view.

#Predictive analytics based on prior orders

#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 
#Singularize the db_operations to reference in both